In [133]:
from ACLScheduleTest.Simulate import Simulator
from ACLScheduleTest.Analysis import Analyst
from ACLScheduleTest.data import RandomDsk
import numpy as np
from time import time

rd = RandomDsk(
    mute=True, node_num=100,
    st_node_num=10,
    min_time=1, time_lv=0, time_rv=0, min_time_step=0,
    avg_mem_peek=1, mem_pv=0, mem_sv=1
)
dsk = rd.build()
sim = Simulator()

In [134]:
num_workers, num_nodes = 10, 100

information = np.ones((num_nodes, num_workers))
def topoOrder(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
    topoQueue = []
    matrix = np.zeros((num_nodes, num_nodes))
    nodeList = list(dsk.keys())
    for idx, node in enumerate(nodeList):
        for n in dsk[node][1:]:
            if n in nodeList:
                id = nodeList.index(n)
                matrix[idx, id] = 1 # idx depends on id
    while len(topoQueue) < len(nodeList):
        for idx, node in enumerate(nodeList):
            if node in topoQueue:
                continue
            if matrix[idx].sum() == 0:
                topoQueue.append(node)
                matrix[:, idx] = 0
    return topoQueue


## Ant as Agent

In [155]:
current_worker_id = 0
iter = 0
from schedule_func_backup import schedule_out_func
def search():
    global current_worker_id, iter
    current_worker_id, iter = 0, 0
    worker_schedules = []
    for idx in range(num_workers):
        worker_schedule = []
        worker_schedules.append(worker_schedule)
    for node_id in range(num_nodes):
        prob = information[node_id] / information[node_id].sum()
        select_worker = np.random.choice(range(num_workers), p=prob)
        worker_schedules[select_worker].append(node_id)
    print(worker_schedules)
    def search_n_func(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
        global current_worker_id, iter
        if len(worker_schedules[current_worker_id])==0:
            return None
        node_id = worker_schedules[current_worker_id][0]
        node = f'node{node_id}'

        worker_schedules[current_worker_id].remove(node_id)
        current_worker_id += 1
        if current_worker_id >= num_workers:
            current_worker_id = 0
            iter += 1
            print(f'\niter {iter}')
        #print(current_worker_id)
        nodeList = set(dsk.keys())
        #print(node)

        pre_nodes = [arg for arg in dsk[node][1:] if arg in nodeList]
        for arg in pre_nodes:
            if arg in nodeList and arg not in cache.keys():
                print('None')
                return None
        print(node)
        return node
    result = sim.simulate(search_n_func, schedule_out_func, mem_bound=50, worker_bound=10, rd=rd)
    print(result)

## Train

In [157]:
def train(num_epochs, num_agent):
    for epoch in range(num_epochs):
        for agent_idx in range(num_agent):
            search()
train(1,1)

[[5, 26, 27, 28, 32, 33, 36, 40, 49, 51, 52, 66, 84, 89, 98], [12, 18, 22, 42, 45, 59, 67, 93, 99], [2, 6, 15, 29, 38, 44, 72, 73, 86, 91, 92], [13, 30, 47, 57, 77, 79, 85, 95], [1, 8, 9, 11, 14, 16, 25, 37, 61, 63, 88, 94], [31, 65, 69, 70, 80, 81], [7, 20, 23, 55, 56, 64, 75, 78, 82, 83, 87, 90], [3, 34, 41, 43, 50, 53, 54, 74, 97], [0, 4, 10, 39, 46, 48, 62, 68, 71], [17, 19, 21, 24, 35, 58, 60, 76, 96]]
node5
None
node2
None
node1
None
node7
node3
node0

iter 1
None
None
None
node6
None
node8
None
None
None
node4

iter 2
None
None
node22
None
None
node9
None
node23
node41
node10

iter 3
node21
None
None
None
None
node11
node70
None
None
None

iter 4
None
node32
node45
node38
None
node14
None
None
None
None

iter 5
None
None
None
node44
None
node16
None
None
None
None

iter 6
None
None
None
None
None
None
({'worker_0': None, 'worker_1': None, 'worker_2': None, 'worker_3': None, 'worker_4': None, 'worker_5': None, 'worker_6': None, 'worker_7': None, 'worker_8': None, 'worker_9': None

## schedule_in_func 

## schedule_out_func